# Area Map 


https://scitools.org.uk/cartopy/docs/v0.5/matplotlib/introductory_examples/02.polygon.html

https://tc.copernicus.org/articles/15/4675/2021/tc-15-4675-2021.pdf

In [ ]:
import cartopy 
import numpy as np 
import xarray as xr
import geopandas as gpd
import contextily as ctx
import cartopy.crs as ccrs
import matplotlib.pyplot as plt 
import matplotlib.path   as mpath
import matplotlib.colors as colors
from matplotlib.patches import Rectangle

class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        super().__init__(vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        # Note also that we must extrapolate beyond vmin/vmax
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1.]
        return np.ma.masked_array(np.interp(value, x, y,
                                            left=-np.inf, right=np.inf))

    def inverse(self, value):
        y, x = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.interp(value, x, y, left=-np.inf, right=np.inf)
    
    
def make_boundary(): 
    """
    https://scitools.org.uk/cartopy/docs/latest/gallery/lines_and_polygons/always_circular_stereo.html
    https://matplotlib.org/stable/api/path_api.html#matplotlib.path.Path
    """
    
    # Create vertices of for the unit circle
    theta = np.linspace(0, 2*np.pi, 201)
    center, radius = [0.0, 0.0], 1.0
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = verts * radius + center

    # extract the first quadrant from the unit circle
    quad1 = circle[:51]
    edges = (np.linspace(0.0,1.0,25) * np.array([[0],[1]])).T 
    path  = np.vstack((quad1, edges))

    # Make a path object
    boundary = mpath.Path(path)
    
    return boundary

In [ ]:
RGI_IDS = ["01.16121",    # Tweedsmuir Glacier
           "01.16122",    # Fisher     Glacier
           "01.16262",    # Spring     Glacier
           "01.17183",    # Klutlan    Glacier
           "01.16198-LK", # Kluane     Glacier
           "01.16195",    # South      Glacier
           "01.16855",    # Glacier #18 from Crompton studies
           "01.16732"]    # Glacier #13 from Crompton studies

# File-path to RGI outlines
RGI_FP = "../../../Desktop/Big_Glaciers/Glaciers/RGI60-{}/GLIMS/RGI60-{}.shp"

In [ ]:
nc_fp = "../input_data/mass_balance/Kaskawulsh_NetBalance.nc"

with xr.open_dataset(nc_fp) as MB_new: 
    pass

midnorm = MidpointNormalize(vmin=-10., vcenter=0, vmax=5)

In [ ]:
extent = [-141.75, -137.75, 59.65, 61.75]
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

# Coordinate Reference System
CRS = ccrs.AlbersEqualArea(central_lon, central_lat)
# Base image source 
source = ctx.providers.Esri.WorldImagery

province = cartopy.feature.NaturalEarthFeature(
    category="cultural", scale="50m", name="admin_1_states_provinces_lines"
)
    
fig = plt.figure(figsize=(9, 6), constrained_layout=True)
ax  = plt.axes(projection=CRS)

boundary = make_boundary()
ax.set_boundary(boundary, transform=ax.transAxes)
ax.set_extent(extent)

# Add the base image
ctx.add_basemap(ax, 
                source=source, 
                crs=CRS.proj4_init,
                attribution_size=6 )

ax.add_feature(cartopy.feature.BORDERS, edgecolor='black')
ax.add_feature(province, linestyle="-", edgecolor="k", facecolor='None')


im = ax.pcolormesh(MB_new.lon, 
              MB_new.lat[::-1,:], 
              MB_new.B, 
              transform=ccrs.epsg(32607),
              cmap='RdBu', 
              norm=midnorm)

for rgi_id in RGI_IDS: 
    split = rgi_id.split("-")[0]
    fp  = RGI_FP.format(rgi_id, split)
    shp = gpd.read_file(fp)
    shp.to_crs(crs=CRS.proj4_init).plot(ax=ax,facecolor=None,edgecolor='k',linewidth=0.5,alpha=0.75)

    
ax.add_patch(Rectangle(xy=[-139.25, 61], width=1.0, height=1.0, facecolor='red', transform=CRS))

    
fig.colorbar(im, ax=[ax], location='left', label='Mass Balance m / yr')
fig.savefig('/Users/andrewnolan/Desktop/AreaMap.png', dpi=300, bbox_inches='tight', facecolor='w')

In [ ]:
# #!/usr/bin/env python

# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy
# import cartopy.crs as ccrs  # projections

# WATERCOLOR = '#7AA1DA'

# #This region in the phillipines causes no errors
# #xmin,xmax,ymin,ymax = (122.460700,128.460700,6.967600,12.877600)

# #this region in india causes an error
# xmin,xmax,ymin,ymax = (90.6505, 96.6505, 22.080346634631709, 27.526853365368289)


# clon = (xmin + xmax)/2
# clat = (ymin + ymax)/2
# width = (xmax-xmin)*np.cos(np.radians(clat))*111.191
# height = (ymax-ymin)*111.191
# aspect = width/height
# FIGWIDTH = 7.0
# figheight = FIGWIDTH/aspect
# bbox = (xmin,ymin,xmax,ymax)
# bounds = (xmin,xmax,ymin,ymax)
# figsize = (FIGWIDTH,figheight)

# proj = ccrs.Mercator(central_longitude=clon,
#                      min_latitude=ymin,
#                      max_latitude=ymax,
#                      globe=None)
# geoproj = ccrs.PlateCarree()

# figure = plt.figure(figsize=figsize)
# ax = figure.add_axes([0,0,1,1],projection=proj)
# ax.set_extent([xmin, xmax, ymin, ymax],crs=geoproj)

# #normally I would do interesting things here, but they're not germane to 
# #reproducing the problem

# corner_rect = (0.84999999999999987,
#                0.012441587723185982,
#                0.14000000000000001,
#                0.13930240350766115)

# clat2 = round(clat)
# clon2 = round(clon)

# #draw a little globe in the corner showing in small-scale where the earthquake is located.
# proj = ccrs.Orthographic(central_latitude=clat2,
#                          central_longitude=clon2)

# ax2 = figure.add_axes(corner_rect,projection=proj)

# #in at least one area in the world (India), this raises an exception once you draw or save
# ax2.add_feature(cartopy.feature.OCEAN, zorder=0,facecolor=WATERCOLOR,edgecolor=WATERCOLOR)

In [ ]:
2300 / 60 